In [1]:
import requests
def fetchData():
    api_url = "https://api.uiucbus.com/api/getdeparturesbystop?stop_id=1STDAN"
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return ""
data = fetchData()
print(data["time"], data["departures"][1])

2025-12-19T18:46:31-06:00 {'stop_id': '1STDAN:1', 'headsign': '22S Illini', 'route': {'route_color': '5a1d5a', 'route_id': 'ILLINI', 'route_long_name': 'Illini', 'route_short_name': '22', 'route_text_color': 'ffffff'}, 'trip': {'trip_id': '[@7.0.41200832@][2][1238430887312]/204__I7_UIF', 'trip_headsign': 'PAR', 'route_id': 'ILLINI', 'block_id': 'I7 UIF', 'direction': 'South', 'service_id': 'I7 UIF', 'shape_id': '22S ILLINI 20'}, 'vehicle_id': '2105', 'origin': {'stop_id': 'LNCLNKLRNY:1'}, 'destination': {'stop_id': 'PAR:2'}, 'is_monitored': True, 'is_scheduled': True, 'is_istop': True, 'scheduled': '2025-12-19T18:57:11-06:00', 'expected': '2025-12-19T18:56:36-06:00', 'expected_mins': 10, 'location': {'lat': 40.111617, 'lon': -88.224033}}


In [2]:
print("---DATA DICTIONARY---")
dataKeys = data.keys()
for key in dataKeys:
    print(f"{key}: TYPE: {type(data[key])} , EX: {data[key]}")

---DATA DICTIONARY---
time: TYPE: <class 'str'> , EX: 2025-12-19T18:46:31-06:00
new_changeset: TYPE: <class 'bool'> , EX: True
status: TYPE: <class 'dict'> , EX: {'code': 200, 'msg': 'ok'}
rqst: TYPE: <class 'dict'> , EX: {'method': 'GetDeparturesByStop', 'params': {'pt': 60, 'stop_id': '1STDAN'}}
departures: TYPE: <class 'list'> , EX: [{'stop_id': '1STDAN:6', 'headsign': '22N Illini', 'route': {'route_color': '5a1d5a', 'route_id': 'ILLINI', 'route_long_name': 'Illini', 'route_short_name': '22', 'route_text_color': 'ffffff'}, 'trip': {'trip_id': '[@7.0.41200832@][1][1238430123625]/71__I6_UIMF', 'trip_headsign': 'Lincoln & Killarney', 'route_id': 'ILLINI', 'block_id': 'I6 UIMF', 'direction': 'North', 'service_id': 'I6 UIMF', 'shape_id': '22N ILLINI 10'}, 'vehicle_id': '2104', 'origin': {'stop_id': 'PAR:2'}, 'destination': {'stop_id': 'LNCLNKLRNY:1'}, 'is_monitored': True, 'is_scheduled': True, 'is_istop': True, 'scheduled': '2025-12-19T18:54:02-06:00', 'expected': '2025-12-19T18:55:05-0

In [3]:
print("---DEPARTURE DICTIONARY---")
departures = data.get("departures", [])
depKeys = departures[0].keys()
for k in depKeys:
    print(f"{k}: type: {type(departures[0][k])}, ex: {departures[0][k]}")

---DEPARTURE DICTIONARY---
stop_id: type: <class 'str'>, ex: 1STDAN:6
headsign: type: <class 'str'>, ex: 22N Illini
route: type: <class 'dict'>, ex: {'route_color': '5a1d5a', 'route_id': 'ILLINI', 'route_long_name': 'Illini', 'route_short_name': '22', 'route_text_color': 'ffffff'}
trip: type: <class 'dict'>, ex: {'trip_id': '[@7.0.41200832@][1][1238430123625]/71__I6_UIMF', 'trip_headsign': 'Lincoln & Killarney', 'route_id': 'ILLINI', 'block_id': 'I6 UIMF', 'direction': 'North', 'service_id': 'I6 UIMF', 'shape_id': '22N ILLINI 10'}
vehicle_id: type: <class 'str'>, ex: 2104
origin: type: <class 'dict'>, ex: {'stop_id': 'PAR:2'}
destination: type: <class 'dict'>, ex: {'stop_id': 'LNCLNKLRNY:1'}
is_monitored: type: <class 'bool'>, ex: True
is_scheduled: type: <class 'bool'>, ex: True
is_istop: type: <class 'bool'>, ex: True
scheduled: type: <class 'str'>, ex: 2025-12-19T18:54:02-06:00
expected: type: <class 'str'>, ex: 2025-12-19T18:55:05-06:00
expected_mins: type: <class 'int'>, ex: 9
loc

In [4]:
# Expects Data dictionary from fetchData()
# Returns a list of dictionaries with headsign and expected time info
def fetchBusInfoFromData(data):
    departures = data.get("departures", [])
    out = []
    
    # If no busses, return empty list
    if not departures: 
        return out
    
    # For every bus get the headsign and expected time (split into multiple fields)
    for busInfo in departures:
        busDict = {}
        busDict["headsign"] = busInfo.get("headsign", "")
        expString = busInfo.get("expected", "")
        if expString == "":
            continue
        expStringHMS = expString[-14:-6]
        busDict["expectedHMS"] = expStringHMS
        busDict["expectedH"] = int(expStringHMS[:2])
        busDict["expectedM"] = int(expStringHMS[3:5])
        busDict["expectedS"] = int(expStringHMS[6:8])
        out.append(busDict)
    
    return out

# Expects Data dictionary from fetchData()
# Returns a dictionary with current time info
def getCurTime(data):
    timeStr = data.get("time", {})
    timeDict = {}
    if not timeStr:
        return timeDict
    timeStrHMS = timeStr[-14:-6]
    timeDict["timeStrHMS"] = timeStrHMS
    timeDict["hour"] = int(timeStrHMS[:2])
    timeDict["min"] = int(timeStrHMS[3:5])
    timeDict["sec"] = int(timeStrHMS[6:8])
    return timeDict

busList = fetchBusInfoFromData(data)
for b in busList:
    print(b)
# print(getCurTime(data))

{'headsign': '22N Illini', 'expectedHMS': '18:55:05', 'expectedH': 18, 'expectedM': 55, 'expectedS': 5}
{'headsign': '22S Illini', 'expectedHMS': '18:56:36', 'expectedH': 18, 'expectedM': 56, 'expectedS': 36}
{'headsign': '22N Illini', 'expectedHMS': '19:04:02', 'expectedH': 19, 'expectedM': 4, 'expectedS': 2}
{'headsign': '22S Illini', 'expectedHMS': '19:07:11', 'expectedH': 19, 'expectedM': 7, 'expectedS': 11}
{'headsign': '14E Navy', 'expectedHMS': '19:10:40', 'expectedH': 19, 'expectedM': 10, 'expectedS': 40}
{'headsign': '22N Illini', 'expectedHMS': '19:14:02', 'expectedH': 19, 'expectedM': 14, 'expectedS': 2}
{'headsign': '22S Illini', 'expectedHMS': '19:17:11', 'expectedH': 19, 'expectedM': 17, 'expectedS': 11}
{'headsign': '220N Illini', 'expectedHMS': '19:24:00', 'expectedH': 19, 'expectedM': 24, 'expectedS': 0}
{'headsign': '22S Illini', 'expectedHMS': '19:27:11', 'expectedH': 19, 'expectedM': 27, 'expectedS': 11}
{'headsign': '220N Illini', 'expectedHMS': '19:34:00', 'expect

In [5]:
## Timing check
for i in range(10):
    d = fetchData()
    i = fetchBusInfoFromData(d)
    t = getCurTime(d)
    i, d, t = None, None, None
# Result: Averages 0.25 seconds for fetch and parse.

In [6]:
import random
import numpy as np
# mat2 = [ [0, 1, 1, 1, 0],
#         [1, 0, 0, 0, 1],
#         [0, 0, 0, 1, 0],
#         [0, 0, 1, 0, 0],
#         [0, 1, 0, 0, 0],
#         [1, 1, 1, 1, 1,]]
# spacer = np.zeros((6,1))

# Print it
def printMatrix(matrix):
    for row in matrix:
        line = ''.join('█' if cell else ' ' for cell in row)
        print(line)

# mat3 = np.concatenate((mat2, spacer, mat2), axis=1)
# printMatrix(mat3)

In [12]:
charToMatrix = {
    " " : np.zeros((6,1)),
    "0" : np.array([[0, 1, 1, 1, 0],
                    [1, 0, 0, 1, 1],
                    [1, 0, 1, 0, 1],
                    [1, 1, 0, 0, 1],
                    [1, 0, 0, 0, 1],
                    [0, 1, 1, 1, 0,]]),
    "1" : np.array([[0, 1, 1, 0, 0],
                    [1, 0, 1, 0, 0],
                    [0, 0, 1, 0, 0],
                    [0, 0, 1, 0, 0],
                    [0, 0, 1, 0, 0],
                    [1, 1, 1, 1, 1,]]),
    "2" : np.array([[1, 1, 1, 1, 0],
                    [0, 0, 0, 0, 1],
                    [0, 1, 1, 1, 0],
                    [1, 0, 0, 0, 0],
                    [1, 0, 0, 0, 0],
                    [0, 1, 1, 1, 1,]]),
    "3" : np.array([[1, 1, 1, 1, 0],
                    [0, 0, 0, 0, 1],
                    [0, 1, 1, 1, 0],
                    [0, 0, 0, 0, 1],
                    [0, 0, 0, 0, 1],
                    [1, 1, 1, 1, 0]]),
    "4" : np.array([[1, 0, 0, 1, 0],
                    [1, 0, 0, 1, 0],
                    [1, 1, 1, 1, 1],
                    [0, 0, 0, 1, 0],
                    [0, 0, 0, 1, 0],
                    [0, 0, 0, 1, 0]]),
    "5" : np.array([[1, 1, 1, 1, 1],
                    [1, 0, 0, 0, 0],
                    [1, 1, 1, 1, 0],
                    [0, 0, 0, 0, 1],
                    [0, 0, 0, 0, 1],
                    [1, 1, 1, 1, 0]]),
    "6" : np.array([[1, 1, 1, 1, 1],
                    [1, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1],
                    [1, 0, 0, 0, 1],
                    [1, 0, 0, 0, 1],
                    [1, 1, 1, 1, 1]]),
    "7" : np.array([[1, 1, 1, 1, 1],
                    [0, 0, 0, 0, 1],
                    [0, 0, 0, 1, 0],
                    [0, 0, 1, 0, 0],
                    [0, 1, 0, 0, 0],
                    [1, 0, 0, 0, 0]]),
    "8" : np.array([[0, 1, 1, 1, 0],
                    [1, 0, 0, 0, 1],
                    [0, 1, 1, 1, 0],
                    [1, 0, 0, 0, 1],
                    [1, 0, 0, 0, 1],
                    [0, 1, 1, 1, 0]]),
    "9" : np.array([[0, 1, 1, 1, 0],
                    [1, 0, 0, 0, 1],
                    [0, 1, 1, 1, 1],
                    [0, 0, 0, 0, 1],
                    [0, 0, 0, 0, 1],
                    [0, 0, 0, 0, 1]]),
    ":" : np.array([[0, 0, 0],
                    [0, 1, 0],
                    [0, 0, 0],
                    [0, 0, 0],
                    [0, 1, 0],
                    [0, 0, 0]]),
    "N" : np.array([[1, 1, 0, 0, 1],
                    [1, 1, 0, 0, 1],
                    [1, 0, 1, 0, 1],
                    [1, 0, 1, 0, 1],
                    [1, 0, 0, 1, 1],
                    [1, 0, 0, 1, 1]]),
    "S" : np.array([[0, 1, 1, 1, 0],
                    [1, 0, 0, 0, 1],
                    [0, 1, 1, 0, 0],
                    [0, 0, 0, 1, 0],
                    [1, 0, 0, 0, 1],
                    [0, 1, 1, 1, 0]]),
    "F" : np.array([[1, 1, 1, 1, 1],
                    [1, 0, 0, 0, 0],
                    [1, 1, 1, 1, 0],
                    [1, 0, 0, 0, 0],
                    [1, 0, 0, 0, 0],
                    [1, 0, 0, 0, 0]]),
    "C" : np.array([[0, 1, 1, 1, 1],
                    [1, 0, 0, 0, 0],
                    [1, 0, 0, 0, 0],
                    [1, 0, 0, 0, 0],
                    [1, 0, 0, 0, 0],
                    [0, 1, 1, 1, 1]]),
    "/" : np.array([[0, 0, 1],
                    [0, 0, 1],
                    [0, 1, 0],
                    [0, 1, 0],
                    [1, 0, 0],
                    [1, 0, 0]]),
    "E" : np.array([[1, 1, 1, 1, 1],
                    [1, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1],
                    [1, 0, 0, 0, 0],
                    [1, 0, 0, 0, 0],
                    [1, 1, 1, 1, 1]]),
    "W" : np.array([[1, 0, 0, 0, 1],
                    [1, 0, 0, 0, 1],
                    [1, 0, 0, 0, 1],
                    [1, 0, 1, 0, 1],
                    [1, 1, 0, 1, 1],
                    [1, 0, 0, 0, 1]]),
    "." : np.array([[0, 1, 0],
                    [1, 0, 1],
                    [0, 1, 0],
                    [0, 0, 0],
                    [0, 0, 0],
                    [0, 0, 0]]),

    "_" : np.array([[0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0]]),
}

In [13]:
def strToMatrix(input):
    chars = list(input)
    matricies = []
    spacer = np.zeros((6,1))
    for c in chars:
        matricies.append(charToMatrix.get(c, spacer))
        matricies.append(spacer)
    matricies.pop()  # Remove last spacer
    finalMatrix = np.concatenate(matricies, axis=1)
    return finalMatrix

outMat = strToMatrix("123456789")
printMatrix(outMat)
outMat = strToMatrix("NS_:F.CEW /")
printMatrix(outMat)

 ██   ████  ████  █  █  █████ █████ █████  ███   ███ 
█ █       █     █ █  █  █     █         █ █   █ █   █
  █    ███   ███  █████ ████  █████    █   ███   ████
  █   █         █    █      █ █   █   █   █   █     █
  █   █         █    █      █ █   █  █    █   █     █
█████  ████ ████     █  ████  █████ █      ███      █
██  █  ███            █████  █   ████ █████ █   █     █
██  █ █   █        █  █     █ █ █     █     █   █     █
█ █ █  ██             ████   █  █     █████ █   █    █ 
█ █ █    █            █         █     █     █ █ █    █ 
█  ██ █   █        █  █         █     █     ██ ██   █  
█  ██  ███            █          ████ █████ █   █   █  


In [9]:
pip install pyweatherbit


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# https://weather.com/weather/today/l/be3c5ddfd5472f5cd0f1b172069e6f38ddb6e14cfcca4c2688252e1c7d94c4e6

In [1]:
pip install dotenv

  Obtaining dependency information for dotenv from https://files.pythonhosted.org/packages/b2/b7/545d2c10c1fc15e48653c91efde329a790f2eecfbbf2bd16003b5db2bab0/dotenv-0.9.9-py2.py3-none-any.whl.metadata
  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/14/1b/a298b06749107c305e1fe0f814c6c74aea7b2f1e10989cb30f544a1b3253/python_dotenv-1.2.1-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
import requests
# from dotenv import load_dotenv
# load_dotenv()
api_url = 'http://api.openweathermap.org/data/2.5/weather'
appid = os.getenv("API_KEY") 
r = requests.get(url=api_url, params=dict(q='Champaign', APPID=appid))


In [11]:
K = (r.json())["main"]["feels_like"]
((K - 273.15) * 9/5) + 32

KeyError: 'main'

In [ ]:
from env import W_KEY
import requests

def fetchTempAvg():
    api_url = 'http://api.openweathermap.org/data/2.5/weather'
    appid = W_KEY
    r = requests.get(url=api_url, params=dict(q='Champaign', APPID=appid))
    return r.json()
    Ka = (r.json())["main"]["temp_min"]
    Kb = (r.json())["main"]["temp_max"]
    K = Kb
    F = ((K - 273.15) * 9/5) + 32
    return int(F)
fetchTempAvg()

{'coord': {'lon': -88.2001, 'lat': 40.1334},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': 262.94,
  'feels_like': 255.94,
  'temp_min': 261.97,
  'temp_max': 263.73,
  'pressure': 1029,
  'humidity': 52,
  'sea_level': 1029,
  'grnd_level': 1000},
 'visibility': 10000,
 'wind': {'speed': 9.77, 'deg': 270, 'gust': 13.89},
 'clouds': {'all': 0},
 'dt': 1768858730,
 'sys': {'type': 2,
  'id': 2004863,
  'country': 'US',
  'sunrise': 1768828270,
  'sunset': 1768863332},
 'timezone': -21600,
 'id': 4887163,
 'name': 'Champaign',
 'cod': 200}

In [17]:
import requests
from datetime import datetime, time
import os
from env import W_KEY
import busData
def fetchDataG():
    api_url = "https://api.uiucbus.com/api/getdeparturesbystop?stop_id=GRN2ND"
    response = requests.get(api_url)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return ""
print(fetchDataG())

{'time': '2026-01-19T16:13:45-06:00', 'new_changeset': True, 'status': {'code': 200, 'msg': 'ok'}, 'rqst': {'method': 'GetDeparturesByStop', 'params': {'pt': 60, 'stop_id': 'GRN2ND'}}, 'departures': [{'stop_id': 'GRN2ND:1', 'headsign': '5W Green', 'route': {'route_color': '008063', 'route_id': 'GREEN', 'route_long_name': 'Green', 'route_short_name': '5', 'route_text_color': 'ffffff'}, 'trip': {'trip_id': '8GN511__GN7_UI_MTH', 'trip_headsign': 'Round Barn Road - Holiday Park', 'route_id': 'GREEN', 'block_id': 'GN7 UI MTH', 'direction': 'West', 'service_id': 'GN7 UI MTH', 'shape_id': '5W-PM'}, 'vehicle_id': '2203', 'origin': {'stop_id': 'MNBRDY:2'}, 'destination': {'stop_id': 'WSFLDLWNDL:4'}, 'is_monitored': True, 'is_scheduled': True, 'is_istop': False, 'scheduled': '2026-01-19T16:16:20-06:00', 'expected': '2026-01-19T16:17:43-06:00', 'expected_mins': 4, 'location': {'lat': 40.110533, 'lon': -88.224967}}, {'stop_id': 'GRN2ND:3', 'headsign': '5E Green Hopper', 'route': {'route_color': '0